<a href="https://colab.research.google.com/github/saitejasri1/Shared-ML-project/blob/main/PaddleOCR-final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from paddleocr import PaddleOCR
from PIL import Image
import numpy as np
from googletrans import Translator
from langdetect import detect
import logging

# Suppress debug messages from ppocr
logging.getLogger("ppocr").setLevel(logging.WARNING)
logging.getLogger("ppocr").setLevel(logging.ERROR)

def detect_ingredients_from_image(image_path):
    """
    Perform OCR on the input image file to detect ingredients.

    Args:
    image_path (str): Path to the image file.

    Returns:
    set: Set of detected ingredients.
    """
    # Setup OCR model with English language
    ocr_model = PaddleOCR(lang='en')

    # Check if the image is in HEIC format
    if image_path.lower().endswith('.heic'):
        # Convert HEIC image to PNG format
        heic_img = Image.open(image_path)
        image_np = np.array(heic_img.convert('RGB'))
    else:
        # Load the image
        image = Image.open(image_path)
        # Convert image to numpy array
        image_np = np.array(image)

    # Perform OCR on the image
    result = ocr_model.ocr(image_np)

    # Extract text from OCR result
    text = ""
    if result is not None:
        for line in result:
            for word in line:
                text += word[1][0] + ' '
    else:
        print("No text detected in image:", image_path)

    # Check if the detected text is in English
    if detect(text) != 'en':
        # Translate text to English
        translator = Translator()
        translated = translator.translate(text, src='auto', dest='en')
        text = translated.text

    # Define the list of ingredients
    ingredients = {'beans', 'salt', 'butter', 'sugar', 'onion', 'water', 'eggs', 'oliveoil', 'flour', 'milk',
                   'garliccloves', 'pepper', 'brownsugar', 'garlic', 'all-purposeflour', 'bakingpowder', 'egg',
                   'saltandpepper', 'parmesancheese', 'lemonjuice', 'bakingsoda', 'vegetableoil', 'vanilla',
                   'blackpepper', 'cinnamon', 'tomatoes', 'sourcream', 'garlicpowder', 'vanillaextract', 'oil',
                   'honey', 'onions', 'creamcheese', 'garlicclove', 'celery', 'cheddarcheese', 'unsaltedbutter',
                   'soysauce'}

    # Convert text to lowercase and split into words
    text = text.lower().split()

    # Find intersection of detected ingredients and predefined ingredients list
    detected_ingredients = ingredients.intersection(text)

    return detected_ingredients


# Example usage:
image_path = "/content/drive/MyDrive/do-not-delete/ML-Project/Images/beans.jpg"
detected_ingredients = detect_ingredients_from_image(image_path)
print(detected_ingredients)


{'beans'}


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install paddlepaddle-gpu==2.3.0 -i https://mirror.baidu.com/pypi/simple
!pip install paddleocr
# git clone https://github.com/PaddlePaddle/PaddleOCR
!pip install PaddleOCR
!pip install langdetect
!pip install paddlepaddle

#German to English

!pip install googletrans==4.0.0-rc1

In [ ]:
!pip install paddle

In [ ]:
!pip install paddleocr
!pip install googletrans
!pip install langdetect